# Sentiment Analysis for Stock Market Prediction #

Reference: https://towardsdatascience.com/sentiment-analysis-for-stock-price-prediction-in-python-bed40c65d178

### Imports and Packages ###

In [1]:
import flair
import re

sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2021-10-24 19:26:12,914 loading file /Users/aj/.flair/models/sentiment-en-mix-distillbert_4.pt


In [2]:
sentence = flair.data.Sentence("tesla will go down")
sentiment_model.predict(sentence)
probability = sentence.labels[0].score  # numerical value 0-1
sentiment = sentence.labels[0].value  # 'POSITIVE' or 'NEGATIVE'
print(sentiment)
print(probability)

NEGATIVE
0.9824783205986023


In [4]:
import requests

BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAF8pUQEAAAAAL7QXs9hTVl4Zq2GE%2FYhVQeJ9kyE%3DUaV8oJRddqkRSZfDXYqI7qS2lBnfRV8lj2S0K6BPJeg4pPfRQf'

params = {
    'q': 'tesla',
    'tweet_mode': 'extended',
    'lang': 'en',
    'count': '100'
}

response = requests.get(
    'https://api.twitter.com/1.1/search/tweets.json',
    params=params,
    headers={
        'authorization': 'Bearer '+ BEARER_TOKEN
})

json = response.json()

In [5]:
json

{'statuses': [{'created_at': 'Sun Oct 24 23:26:15 +0000 2021',
   'id': 1452416176529842183,
   'id_str': '1452416176529842183',
   'full_text': '@ShibaInuHodler @Tesla Nooo we dont need that guy, stop tweeting about elon you going to screw the shiba market...',
   'truncated': False,
   'display_text_range': [23, 114],
   'entities': {'hashtags': [],
    'symbols': [],
    'user_mentions': [{'screen_name': 'ShibaInuHodler',
      'name': 'Shiba Inu',
      'id': 1411587179986489346,
      'id_str': '1411587179986489346',
      'indices': [0, 15]},
     {'screen_name': 'Tesla',
      'name': 'Tesla',
      'id': 13298072,
      'id_str': '13298072',
      'indices': [16, 22]}],
    'urls': []},
   'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
   'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
   'in_reply_to_status_id': 1452313590267002886,
   'in_reply_to_status_id_str': '1452313590267002886',
   'in_reply_to_user_i